In [0]:
import nltk
import numpy as np
import random
import string 

In [11]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
df=pd.DataFrame()
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset_Chatbot/articles.csv')
df.head(10)

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...
5,"Em encontro, Bono pergunta a Macri sobre argen...","O vocalista da banda irlandesa U2, Bono, fez u...",2017-09-10,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/10/192...
6,"Posso sair do Brasil quando e como quiser, diz...",O italiano Cesare Battisti disse nesta segunda...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
7,Tite diz querer seguir na seleção após o Mundi...,Pela primeira vez desde que assumiu o comando ...,2017-09-10,esporte,NaN,http://www1.folha.uol.com.br/esporte/2017/10/1...
8,Supremo nega pedido para Senado analisar impea...,O STF (Supremo Tribunal Federal) negou na quin...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
9,"Em teste, WhatsApp Business permite que empres...",O aplicativo de mensagens instantâneas WhatsAp...,2017-09-10,tec,NaN,http://www1.folha.uol.com.br/tec/2017/10/19257...


In [13]:
ilustrada_dataset = df.loc[df.category == 'ilustrada']
ilustrada_dataset.head()

,title,text,date,category,subcategory,link
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
24,"'Sem informação você é manipulado', diz ator M...",No ar como o protagonista da atual novela das ...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
30,Ministro da Cultura nega ter proposto alteraçã...,"O ministro da Cultura, Sérgio Sá Leitão, negou...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
35,Meryl Streep fala sobre produtor acusado de di...,A premiada atriz Meryl Streep falou que as açõ...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...


In [0]:
# Merges the title and text with a separator 
merged_text = [str(title) + ' ---- ' + str(text) for title, text in zip(ilustrada_dataset.title, ilustrada_dataset.text)]

print(merged_text[0])

In [15]:
import io

print("ASSISTENTE: Serei seu Assistente. Se você desejar sair diga tchau!")
  
#Escolha do assunto
assunto = input('Escolha o assunto da notícia desejada: ') 
if assunto == 'ambiente':
   f=io.open('/content/drive/My Drive/chatbot/ambiente.txt','r', encoding="utf-8")
elif assunto == 'cotidiano':
   f=io.open('/content/drive/My Drive/chatbot/cotidiano.txt','r', encoding="utf-8")
elif assunto == 'esporte':
   f=io.open('/content/drive/My Drive/chatbot/esporte.txt','r', encoding='utf8')
elif assunto == 'ilustrada':
   f=io.open('/content/drive/My Drive/chatbot/ilustrada.txt','r', encoding="utf-8")
elif assunto == 'mercado':
   f=io.open('/content/drive/My Drive/chatbot/mercado.txt','r',  encoding="utf-8")
elif assunto == 'mundo':
   f=io.open('/content/drive/My Drive/chatbot/mundo.txt','r', encoding='utf-8')
elif assunto == 'poder':
   f=io.open('/content/drive/My Drive/chatbot/poder.txt','r', encoding='utf-8')
elif assunto == 'tec':
   f=io.open('/content/drive/My Drive/chatbot/tec.txt','r', encoding='utf-8')
else:
   print("Ainda não tenho informações sobre esse assunto")

raw=f.read()
raw=raw.lower() # converts to lowercase
nltk.download('punkt')   
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw) # converts to list of sentences 
word_tokens = nltk.word_tokenize(raw) # converts to list of words
  
#Keyword matching
SAUDACAO_ENTRADA = ("ola", "oi", "bom dia", "boa tarde", "boa noite", "ei")
RESPOSTA_SAUDACAO = ["Oi", "Olá", "Em que posso lhe ajudar?", "Estou feliz por você estar falando comigo!"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in SAUDACAO_ENTRADA:
           return random.choice(RESPOSTA_SAUDACAO)
  
#Generating Response
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Desculpe! Ainda não sei sobre esse assunto."
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

flag=True

#Pre-processing the raw text
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='tchau'):
        if(user_response=='Obrigado' or user_response=='Espero ter ajudado'):
            flag=False
            print("ASSISTENTE: Seja bem vindo!") 
        else:
            if(greeting(user_response)!=None):
                print("ASSISTENTE: "+greeting(user_response))
            else:
                print("ASSISTENTE: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ASSISTENTE: Até logo!")



ASSISTENTE: Serei seu Assistente. Se você desejar sair diga tchau!
Escolha o assunto da notícia desejada: ilustrada
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
ministro
ASSISTENTE: "ministro da cultura nega ter proposto alteração que abriria portas à censura","o ministro da cultura, sérgio sá leitão, negou categoricamente nesta segunda (9) que tenha proposto qualquer alteração na regulamentação da lei rouanet que vete projetos que "vilipendiem a fé religiosa, promovam a sexualização precoce de crianças e adolescentes ou façam apologia a crimes ou atividades criminosas".
atriz
ASSISTENTE: streep ganhou o oscar e o globo de ouro de melhor atriz por "a dama de ferro" (2012), filme produzido por weinstein no qual ela interpretou a premiê britânica margaret thatcher.
meryl
ASSISTENTE: "meryl streep fala sobre pr